# Project 2: Gauss-Jordan
---
- *Name*: **Lê Võ Minh Phương**
- *Student ID*: **22120286**
- *Class*: **Toán ứng dụng thống kê 22_3**
---


# I. TRÌNH BÀY GIẢI THUẬT

**1. Phương pháp Gauss-Jordan:**  
*Note*: Kèm theo vị trí sử dụng các hàm helper.
- Bước 1: Xác định cột đầu tiên không chứa số không: Hàm lặp qua các cột để tìm leading column.
- Bước 2: Hoán đổi dòng đầu tiên với dòng khác đã được đánh dấu ở Bước 1 nếu cần: Hàm sử dụng hoán đổi dòng để đặt một số không ở cột chính lên đầu tiên.
- Bước 3: Điều chỉnh dòng đầu tiên để có phần tử chính là 1: Hàm điều chỉnh dòng đầu tiên để đảm bảo phần tử chính là 1.
- Bước 4: Sử dụng các phép biến đổi sơ cấp hàng cho các hàng phía dưới để biến tất cả các phần tử trong leading column thành số không, bắt đầu từ hàng thứ hai: Hàm áp dụng các phép toán hàng cơ bản để loại bỏ các phần tử dưới phần tử chính trong cột chính hiện tại.
- Bước 5: Bỏ qua dòng đầu tiên và lặp lại từ Bước 1 với ma trận đã rút gọn: Sau mỗi lần lặp, hàng đầu tiên của ma trận được bỏ qua và quá trình được lặp lại với ma trận đã rút gọn.

**2. Thuật toán Tìm Nghịch Đảo của Ma trận:**

- Bước 1: Cho ma trận vuông A kích thước n x n, tìm ma trận nghịch đảo từ ma trận mở rộng $(A|I_n)$: Ma trận cần tìm nghịch đảo được kết hợp với ma trận đơn vị để thực hiện phương pháp Gauss-Jordan.
- Bước 2: Áp dụng thuật toán Gauss-Jordan cho ma trận mở rộng: Hàm `inverse` thực hiện thuật toán Gauss-Jordan với ma trận đã kết hợp.
- Bước 3: Sau khi thu được ma trận mở rộng $(I_n|A^{-1})$, ma trận $(A^{-1})$ là ma trận nghịch đảo của A: Sau khi loại bỏ ma trận đơn vị, ma trận nghịch đảo được trả ra.

In [6]:
import numpy as np
import copy

# II. CÀI ĐẶT CÁC HÀM ĐỂ TÌM MA TRẬN NGHỊCH ĐẢO

##1. Viết hàm nhập ma trận vuông để tìm nghịch đảo

In [7]:
def input_square_matrix():

  n = int(input("Nhập số dòng (cột) của ma trận vuông A: "))
  matrix = []

  for i in range(n):

    row_elements = input(f"Nhập các phần tử của dòng thứ {i + 1} (cách nhau bởi 1 dấu cách): ").split()

    matrix.append([float(element) for element in row_elements])

  return matrix


##2. Sử dụng hàm tính định thức để kiểm tra tính khả nghịch

In [8]:

def createSubMatrix(matrix:list, rowIndex:int, colIndex:int):
    subMatrix = copy.deepcopy(matrix)


    subMatrix = subMatrix[:rowIndex] + subMatrix[rowIndex+1:]


    numRowsSubMatrix = len(subMatrix)
    for i in range(numRowsSubMatrix):
        subMatrix[i] = subMatrix[i][:colIndex] + subMatrix[i][colIndex+1:]

    return subMatrix

# tính định thức cho ma trận vuông
def computeDeterminant(matrix:list):
    # Base case: ma trận chỉ có 1 phần tử
    if len(matrix) == 1 and len(matrix[0]) == 1:
        return matrix[0][0]

    total = 0

    # Loop để tạo các ma trận con
    for colIndex in range(len(matrix[0])):
        subMatrix = createSubMatrix(matrix, 0, colIndex)

        sign = (-1) ** (colIndex)

        # Gọi đệ quy để tính định thức với từng ma trận con
        subDeterminant = computeDeterminant(subMatrix)

        # Tính tổng kết qur theo công thức
        total += sign * matrix[0][colIndex] * subDeterminant

    return total


## 3. Viết hàm đưa về ma trận đơn vị

### 3.1 Hàm tạo ma trận đơn vị có cùng kích thước với ma trận A

In [9]:
def create_identity_matrix(identity_matrix: list, size: int) -> list:
    # Tạo ma trận đơn vị kích thước size x size
    for i in range(size):
        identity_matrix.append([])
        for j in range(size):
            if i == j:
                identity_matrix[i].append(1.0)  # Phần tử trên đường chéo chính bằng 1
            else:
                identity_matrix[i].append(0.0)  # Các phần tử còn lại bằng 0
    return identity_matrix

### 3.2 Viết hàm Kiểm tra xem có giá trị khác 0 ở dưới cột đã cho từ hàng `start_row`

In [10]:
def has_nonzero_below_column(matrix: list, column: int, start_row: int) -> bool:

    for i in range(start_row, len(matrix)):
        if matrix[i][column] != 0:
            return True
    return False

### 3.3 Viết hàm kiểm tra 1 cột có chứa toàn số 0 hay không

In [11]:
def has_nonzero_row(matrix: list, row: int, num_columns: int) -> bool:
    # Kiểm tra xem hàng của ma trận có toàn số 0 sau num_columns cột không
    j = 0
    for entry in matrix[row]:
        if j == num_columns:
            break
        if entry != 0:
            return True
        j += 1
    return False

### 3.4 Viết hàm hoán đổi dòng trong ma trận và nhân một dòng của ma trận với một số vô hướng

In [12]:
def swap_rows(matrix: list, i: int, j: int):
    # Hoán đổi hai hàng của ma trận
    matrix[i], matrix[j] = matrix[j], matrix[i]

def multiply_row(matrix: list, row_index: int, scalar: float):
    # Nhân một hàng của ma trận với một số vô hướng
    for j in range(len(matrix[row_index])):
        value = scalar * matrix[row_index][j]
        if value == -0.0:
            value = 0.0
        matrix[row_index][j] = truncate_number(value, 10)

###  3.5 Viết hàm thêm một hàng của ma trận vào một hàng khác sau khi nhân với một hằng số (`scalar`)

In [13]:
def add_row_to_row(matrix: list, row_index: int, target_row_index: int, scalar: float):

    num_columns = len(matrix[row_index])
    for k in range(num_columns):
        value = matrix[row_index][k] + scalar * matrix[target_row_index][k]
        if value == -0.0:
            value = 0.0
        matrix[row_index][k] = truncate_number(value, 10)

### 3.6 Viết hàm làm tròn số thập phân (lấy 7 chữ số sau dấu chấm thập phân)

In [14]:
def truncate_number(number, decimals=0):
    # Hàm làm tròn số với số thập phân decimals
    multiplier = 10 ** decimals
    return int(number * multiplier) / multiplier


### 3.7 Viết hàm nhận vào 1 ma trận vuông và đưa ra ma trận nghịch đảo của nó (nếu có)

In [15]:
def inverse(matrix: list):
    # Tìm ma trận nghịch đảo của ma trận đã cho bằng phương pháp Gauss-Jordan

    # Khởi tạo ma trận Gauss-Jordan từ ma trận ban đầu
    gauss_jordan_matrix = copy.deepcopy(matrix)
    # Tạo ma trận đơn vị
    identity_matrix = []
    size = len(matrix)
    identity_matrix = create_identity_matrix(identity_matrix, size)

    # Kết hợp ma trận ban đầu với ma trận đơn vị để tạo ma trận mở rộng
    concat_index = 0
    for row in gauss_jordan_matrix:
        for k in range(len(identity_matrix[concat_index])):
            row.append(identity_matrix[concat_index][k])
        concat_index += 1

    num_rows = size
    num_cols = 2 * size

    row = 0
    while row < num_rows:
        col = row

        # Bước 1: Tìm cột dẫn (leading column)
        while col < num_cols and not has_nonzero_below_column(gauss_jordan_matrix, col, row):
            col += 1

        # Nếu không tìm thấy cột dẫn, thoát vòng lặp
        if col == num_cols:
            break

        # Bước 2: Hoán đổi dòng để đưa mục dẫn lên đầu cột
        for pivot_row in range(row, num_rows):
            if gauss_jordan_matrix[pivot_row][col] != 0:
                break
        swap_rows(gauss_jordan_matrix, row, pivot_row)

        # Bước 3: Chia dòng đầu tiên để phần tử chính là 1
        multiply_row(gauss_jordan_matrix, row, 1 / gauss_jordan_matrix[row][col])

        # Bước 4: Sử dụng phép toán hàng cho các hàng phía dưới để biến tất cả phần tử trong cột dẫn thành số 0
        for reversed_row in range(row):
            add_row_to_row(gauss_jordan_matrix, reversed_row, row, -gauss_jordan_matrix[reversed_row][col])

        for k in range(row + 1, size):
            # Kiểm tra hàng có phải hàng không hay không
            if not has_nonzero_row(gauss_jordan_matrix, k, size):
                return ['Ma trận không thể nghịch đảo!']
            # Sử dụng phép toán hàng cho các hàng phía dưới
            add_row_to_row(gauss_jordan_matrix, k, row, -gauss_jordan_matrix[k][col])

        row += 1

    # Loại bỏ cột của ma trận đơn vị khỏi ma trận Gauss-Jordan
    for row in gauss_jordan_matrix:
        for i in range(size):
            del row[0]

    return gauss_jordan_matrix



# III. THỬ NGHIỆM CHƯƠNG TRÌNH

## 1. Lấy dữ liệu từ bàn phím

In [44]:

A = input_square_matrix();
print ("Ma trận A vừa nhập là \n")
for i in A:
  print (i);

#Tính định thức để kiểm tra có nghịch đảo hay không
det = computeDeterminant (A)
print ("Định thức của ma trận A là:", det)

if (det == 0):
  print ("Ma trận không khả nghịch")
else:
  print ("Ma trận nghịch đảo của A là \n")
  A_inverse = inverse(A)
  for i in A_inverse:
    print(i)



Nhập số dòng (cột) của ma trận vuông A: 3
Nhập các phần tử của dòng thứ 1 (cách nhau bởi 1 dấu cách): 1 2 -2
Nhập các phần tử của dòng thứ 2 (cách nhau bởi 1 dấu cách): -2 1 2
Nhập các phần tử của dòng thứ 3 (cách nhau bởi 1 dấu cách): 3 1 -4
Ma trận A vừa nhập là 

[1.0, 2.0, -2.0]
[-2.0, 1.0, 2.0]
[3.0, 1.0, -4.0]
Định thức của ma trận A là: 0.0
Ma trận không khả nghịch


### 1.1 MỞ RỘNG: Sử dụng hàm có sẵn để kiểm tra lại kết quả (`np.linalg.inv()`)

*Note*: cần chuyển ma trận A thành kiểu `np.matrix` để sử dụng, nếu ma trận không khả nghịch thì đưa ra Exception (`LinAlgError: Singular matrix
`), còn nếu khả nghịch thì sẽ xuất ra ma trận khả nghịch tương đương kết quả trên (có sai số ít ở phần thập phân nếu ra số thập phân vô hạn không tuần hoàn)

In [46]:

A_np_matrix = np.matrix(A)
print (A_np_matrix)
try:
  print (np.linalg.inv(A_np_matrix))
except Exception as e:
      print("Xảy ra lỗi khi tính toán ma trận nghịch đảo:", e)


[[ 1.  2. -2.]
 [-2.  1.  2.]
 [ 3.  1. -4.]]
Xảy ra lỗi khi tính toán ma trận nghịch đảo: Singular matrix


### 1.2 Test nhanh với các ma trận có sẵn và kiểm chứng với thư viện có sẵn `np.linalg.inv()`

In [50]:
T1 = [[12.12, 45], [10, 25]]
T1_matrix = np.matrix(T1)

det = computeDeterminant (T1)
print ("Định thức của ma trận là: ", det)

if (det == 0):
  print ("Ma trận không khả nghịch")
else:
  print ("Ma trận nghịch đảo là \n")
  T1 = inverse(T1)
  for i in T1:
    print(i)

print ("Kiểm chứng với hàm có sẵn\n")

for i in np.linalg.inv(T1_matrix):
  print(i)

print ("------------------------------------")

T2 = [[2, -3, -4], [-2,2,3], [-3,4,6] ]
T2_matrix = np.matrix(T2)

det = computeDeterminant (T2)
print ("Định thức của ma trận là: ", det)

if (det == 0):
  print ("Ma trận không khả nghịch")
else:
  print ("Ma trận nghịch đảo là \n")
  T2 = inverse(T2)
  for i in T2:
    print(i)

print ("Kiểm chứng với hàm có sẵn\n")

for i in np.linalg.inv(T2_matrix):
  print(i)

print ("------------------------------------")

T3 = [[2,-3,3], [1,-1,-2], [-5,7,-4]]

T3_matrix = np.matrix(T3)

det = computeDeterminant (T3)
print ("Định thức của ma trận là: ", det)

if (det == 0):
  print ("Ma trận không khả nghịch")
else:
  print ("Ma trận nghịch đảo là \n")
  T3 = inverse(T3)
  for i in T3:
    print(i)

print ("Kiểm chứng với hàm có sẵn\n")
try:
  for i in np.linalg.inv(T3_matrix):
    print(i)
except Exception as e:
      print("Xảy ra lỗi khi tính toán ma trận nghịch đảo:", e)

Định thức của ma trận là:  -147.0
Ma trận nghịch đảo là 

[-0.1700680269, 0.3061224486]
[0.0680272108, -0.0824489795]
Kiểm chứng với hàm có sẵn

[[-0.17006803  0.30612245]]
[[ 0.06802721 -0.08244898]]
------------------------------------
Định thức của ma trận là:  -1
Ma trận nghịch đảo là 

[0.0, -2.0, 1.0]
[-3.0, 0.0, -2.0]
[2.0, -1.0, 2.0]
Kiểm chứng với hàm có sẵn

[[-1.18423789e-15 -2.00000000e+00  1.00000000e+00]]
[[-3.00000000e+00  9.99200722e-16 -2.00000000e+00]]
[[ 2. -1.  2.]]
------------------------------------
Định thức của ma trận là:  0
Ma trận không khả nghịch
Kiểm chứng với hàm có sẵn

[[4.05323966e+15 2.02661983e+15 2.02661983e+15]]
[[3.15251974e+15 1.57625987e+15 1.57625987e+15]]
[[4.50359963e+14 2.25179981e+14 2.25179981e+14]]


**Nhận xét**: ở trường hợp T3, tuy định thức bằng 0 nhưng hàm `np.linalg.inv()` vẫn cho ra kết quả nhưng đều là số dương vô cùng vì nó sử dụng hàm `np.linalg.solve($(A|I_n)$` và cách lưu trữ dấu chấm động khiến giá trị chỉ đạt mức độ gần đúng nên ta có thể hiểu nó không có ma trận nghịch đảo.

# IV. MÔ TẢ VÀ Ý TƯỞNG CỦA CÁC HÀM ĐÃ CÀI ĐẶT


## Hàm `has_nonzero_below_column`:
- Ý tưởng: Kiểm tra xem có giá trị khác 0 ở dưới cột đã cho từ hàng start_row không.
- Input:  
 + `matrix`: Ma trận đang xem xét.  
 + `column`: Chỉ số của cột cần kiểm tra.  
 + `start_row`: Hàng bắt đầu kiểm tra.
- Output:
bool: Trả về True nếu có ít nhất một giá trị khác 0 ở dưới cột đã cho từ hàng start_row, ngược lại trả về False.

## Hàm create_identity_matrix
- Ý tưởng: Tạo ma trận đơn vị với kích thước đã cho.
- Input:  
  + `identity_matrix`: Ma trận đơn vị cần tạo.  
  + `size`: Kích thước của ma trận đơn vị (số hàng = số cột).
- Output: list: Trả về ma trận đơn vị được tạo.  

## Hàm truncate_number
- Ý tưởng: Làm tròn một số với số chữ số thập phân cụ thể.
- Input:
 + `number`: Số cần làm tròn.
  + `decimals`: Số chữ số thập phân.
- Output:
  - float: Trả về số đã được làm tròn.

## Hàm has_nonzero_row
- Ý tưởng: Kiểm tra xem hàng của ma trận có toàn số 0 sau num_columns cột không.
- Input:
   - `matrix`: Ma trận cần kiểm tra.
   - `row` : Hàng cần kiểm tra.  
   
  - `num_columns`: Số cột cần kiểm tra.
- Output: bool: Trả về True nếu hàng của ma trận có toàn số 0 sau num_columns cột, ngược lại trả về False.

## Hàm swap_rows
- Ý tưởng: Hoán đổi hai hàng của ma trận.
- Input:
  - `matrix`: Ma trận chứa hai hàng cần hoán đổi.
  - `i`: Chỉ số hàng thứ nhất.
  - `j`: Chỉ số hàng thứ hai.
- Output: Không có giá trị trả về, hoán vị 2 dòng của ma trận input

## Hàm multiply_row
- Ý tưởng: Nhân một hàng của ma trận với một số vô hướng.
- Input:
  - `matrix`: Ma trận chứa hàng cần nhân.
  - `row_index`: Chỉ số của hàng cần nhân.
  - `scalar`: Số vô hướng để nhân vào hàng.
- Output: Không có giá trị trả về, giá trị trong 1 hàng đã được nhân 1 hằng số

## Hàm add_row_to_row
- Ý tưởng: Thêm một hàng của ma trận vào một hàng khác với một hệ số nhân.
- Input:
  - `matrix`: Ma trận chứa các hàng.
  - `row_index`: Chỉ số của hàng cần thêm.
  - `target_row_index`: Chỉ số của hàng mục tiêu.
  - `scalar`: Hệ số nhân.
- Output: Không có giá trị trả về, giá trị 1 hàng được cập nhật với 1 dòng khác cùng 1 hệ số

## Hàm inverse
- Ý tưởng: Tìm ma trận nghịch đảo của ma trận đã cho bằng phương pháp Gauss-Jordan.
- Input:
  - `matrix`: Ma trận cần tìm nghịch đảo.
  - `size`: Kích thước của ma trận (số hàng = số cột).
- Output: list: Trả về ma trận nghịch đảo.